# Lab 5 BI

> Agregar bloque entrecomillado




In [2]:
!pip install bytewax==0.13.1 river scipy geopy==2.2.0 requests==2.28.1 kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: geographiclib
    Found existing installation: geographiclib 2.0
    Uninstalling geographiclib-2.0:
      Successfully uninstalled geographiclib-2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
   

Instalamos Redpanda

In [3]:
!curl -1sLf 'https://packages.vectorized.io/nzc4ZYQK3WRGd9sy/redpanda/cfg/setup/bash.deb.sh' | sudo -E bash
!sudo apt install redpanda -y

Executing the  setup script for the 'redpanda/redpanda' repository ...

   OK: Checking for required executable 'curl' ...
   OK: Checking for required executable 'apt-get' ...
   OK: Detecting your OS distribution and release using system methods ...
 ^^^^: ... Detected/provided for your OS/distribution, version and architecture:
 >>>>:
 >>>>: ... distro=ubuntu  version=22.04  codename=jammy  arch=x86_64  
 >>>>:
 NOPE: Checking for apt dependency 'apt-transport-https' ...
   OK: Updating apt repository metadata cache ...
   OK: Attempting to install 'apt-transport-https' ...
   OK: Checking for apt dependency 'ca-certificates' ...
   OK: Checking for apt dependency 'gnupg' ...
   OK: Checking for apt signed-by key support ...
   OK: Importing 'redpanda/redpanda' repository GPG keys ...
   OK: Checking if upstream install config is OK ...
   OK: Installing 'redpanda/redpanda' repository via apt ...
   OK: Updating apt repository metadata cache ...
   OK: The repository has been instal

Iniciamos Redpana

In [4]:
import os
import subprocess

In [5]:

env = os.environ.copy()
p = subprocess.Popen('sudo rpk redpanda start', shell=True, env=env)

In [6]:
!rpk redpanda admin brokers list

NODE-ID  NUM-CORES  MEMBERSHIP-STATUS  IS-ALIVE  BROKER-VERSION
0        2          active             true      v24.2.5 - f65c3be191418e914a298a2df97bdb7f279855be


Importamos lo necesario

In [7]:
import re
from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin import NewTopic
from geopy.geocoders import Nominatim
from river import anomaly
from scipy import stats
import subprocess
import bytewax.dataflow as flow
from bytewax.inputs import ManualInputConfig
from bytewax.outputs import KafkaOutputConfig, StdOutputConfig
from bytewax.execution import run_main
from bytewax.window import TumblingWindowConfig, EventClockConfig

In [8]:
def setup_kafka():
    admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])
    topic = NewTopic(name='traffic_data', num_partitions=1, replication_factor=1)
    try:
        admin_client.create_topics([topic])
        print("Topic 'traffic_data' creado exitosamente.")
    except TopicAlreadyExistsError:
        print("El topic 'traffic_data' ya existe. Continuando con el procesamiento.")
    except Exception as e:
        print(f"Error al crear el topic: {e}")

In [9]:
def get_traffic_data(location):
    return f'Tráfico en {location}: {50 + hash(location) % 50} vehículos/min'

In [10]:
def parse_traffic_data(data):
    match = re.search(r'(\d+) vehículos/min', data)
    return int(match.group(1)) if match else 0

In [11]:
def geolocate(location):
    geolocator = Nominatim(user_agent="traffic_monitor")
    return geolocator.geocode(location)

In [12]:
def detect_anomaly(data):
    detector = anomaly.HalfSpaceTrees(
        n_trees=10,
        height=8,
        window_size=256,
        seed=42
    )
    is_anomaly = detector.score_one({'value': data}) > 0.95
    detector = detector.learn_one({'value': data})
    return is_anomaly

In [13]:
def calculate_variation(data):
    return stats.variation(data)

In [14]:
def process_traffic_data(data):
    if isinstance(data, tuple) and len(data) == 2:
        location, traffic = data
    elif isinstance(data, str):
        location, traffic = data.split(":")[0].split("en ")[-1].strip(), data
    else:
        raise ValueError(f"Unexpected data format: {data}")

    geo = geolocate(location)
    traffic_value = parse_traffic_data(traffic)
    is_anomaly = detect_anomaly(traffic_value)
    return {
        'location': location,
        'coords': (geo.latitude, geo.longitude) if geo else None,
        'traffic': traffic_value,
        'is_anomaly': is_anomaly
    }

In [15]:
def input_builder(worker_index, worker_count, resume_state):
    for data in input_data:
        yield None, data


In [16]:
def build_flow():
    flow_builder = flow.Dataflow()
    flow_builder.input("input", ManualInputConfig(input_builder))
    flow_builder.map(lambda x: print(f"Debug: received data {x}") or x)
    flow_builder.map(process_traffic_data)
    flow_builder.capture(StdOutputConfig())
    return flow_builder

In [17]:
def main(locations):
    setup_kafka()

    global input_data
    input_data = [get_traffic_data(location) for location in locations]

    dataflow = build_flow()
    run_main(dataflow)

In [18]:
# Ejemplo de uso
locations = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]
main(locations)

Topic 'traffic_data' creado exitosamente.
Debug: received data Tráfico en New York: 98 vehículos/min
{'location': 'New York', 'coords': (40.7127281, -74.0060152), 'traffic': 98, 'is_anomaly': False}
Debug: received data Tráfico en Los Angeles: 68 vehículos/min
{'location': 'Los Angeles', 'coords': (34.0536909, -118.242766), 'traffic': 68, 'is_anomaly': False}
Debug: received data Tráfico en Chicago: 94 vehículos/min
{'location': 'Chicago', 'coords': (41.8755616, -87.6244212), 'traffic': 94, 'is_anomaly': False}
Debug: received data Tráfico en Houston: 63 vehículos/min
{'location': 'Houston', 'coords': (29.7589382, -95.3676974), 'traffic': 63, 'is_anomaly': False}
Debug: received data Tráfico en Phoenix: 91 vehículos/min
{'location': 'Phoenix', 'coords': (33.4484367, -112.074141), 'traffic': 91, 'is_anomaly': False}
